# Data Modeling Seasonality

Now that we've explored what the seasonality dataset looks like, the goal is to be able to make modifications to our average prediction from the initial regression to account for seasonality of pricing. This basically means that we will make +/- modifications to our average prediction based on the the day that we are projecting the data for. We can also do similar things for months of the year as well as holidays. This will hopefully reduce the residuals because the seasonality of the pricing data would cause some correlation among the residuals (based on time of the year) violating a lot of the OLS assumptions. We use averages as a way to explore seasonality. More advanced seasonality measurements could be used if we had more data over several years (where we could build an ARIMA or SARIMA model).

In [37]:
import pandas as pd
import numpy as np
import matplotlib
import seaborn as sb
import matplotlib.pyplot as plt
import matplotlib.cm as cmx
import matplotlib.colors as colors
from sklearn import linear_model
import sklearn.metrics as metrics
from sklearn.grid_search import GridSearchCV
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression as Lin_Reg
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
import pylab
import scipy.stats as stats
%matplotlib inline
import datetime as dt
from datetime import datetime

### Process Overview:
The general idea behind this analysis is as follows: we aggregate prices by weekday for each listing. Then, we normalize each listing's price by the monday price to find an average multiplier for each listing for each day. Then, for each day we average across all listings to get a final average multiplier for each day. Lastly, we compare these predictions to a subset of the listings.

In [21]:
#Importing Datafile
results_nona = pd.read_csv('../datasets/seasonality_tomodel.csv')
results_multiplier = pd.read_csv('../datasets/seasonality_tomodel.csv')
b=['Mon','Tue','Wed','Thu','Fri','Sat','Sun']
for i in b[1:7]:
    results_multiplier[i] = results_multiplier[i]/results_multiplier['Mon']
results_multiplier['Mon']= 1
b=['Mon','Tue','Wed','Thu','Fri','Sat','Sun']
for i in b[1:7]:
    results_multiplier[i] = results_multiplier[i]/results_multiplier['Mon']
results_multiplier['Mon']= 1
results_multiplier.head(5)

,Mon,Tue,Wed,Thu,Fri,Sat,Sun,listing_id
0,1,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,3604481.0
1,1,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,2949128.0
2,1,0.991826,0.991826,0.999846,1.138965,1.138965,1.00000,4325397.0
3,1,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,4325398.0
4,1,0.991494,0.994952,1.004395,1.015027,1.011263,0.99895,3426149.0


We see that the dataframe now contains a multiplier for each day of the week for each listing. Now we take an average for each day(averaging across all listings) to see an average multiplier value for each day

In [108]:
multiplier = dict.fromkeys(b)
for index,i in enumerate(multiplier):
    multiplier[i]=results_multiplier.mean()[i]
multiplier

{'Fri': 1.0306309142782333,
 'Mon': 1.0,
 'Sat': 1.0306744793102525,
 'Sun': 1.0009269127770359,
 'Thu': 1.0003671343632679,
 'Tue': 0.99980071473169119,
 'Wed': 0.9998361380714712}

In [115]:
b[1:]

['Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

The results are very much in line with what we saw earlier in our seasonality-exploration file. Monday and Tuesday see a slight dip in their prices(99.9%) while Friday and Saturday see a sizable increase in prices (103%). These are thus the numbers we will be using to apply seasonality to the averages from our previous models. Here, we apply it to RidgeCV the best one that we found.

## Predicting prices using our seasonality averages

Now, it is important to test the performance of the averages we arrived at

In [31]:
#Reproducing the RidgeCV code from 3A
# helper function for plotting residual plots
def plot_residual(ax1, ax2, ax3, y_pred, y_real, line_label, title):
    ax1.scatter(y_pred, 
                y_real, 
                color='blue',
                alpha=0.6,
                label=line_label)
    ax1.set_xlabel('Predicted Y') 
    ax1.set_ylabel('Real Y')
    ax1.legend(loc='best')
    ax1.set_title(title)

    ax2.scatter(y_pred,
                y_real - y_pred, 
                color='green',
                marker='x',
                alpha=0.6,
                label='Residual')
    ax2.set_xlabel('Predicted Y')
    ax2.set_ylabel('Residual')
    
    ax2.axhline(y=0, color='black', linewidth=2.0, alpha=0.7, label='y=0')

    ax2.legend(loc='best')
    ax2.set_title('Residual Plot')
    
    ax3.hist(y_real - y_pred, bins=30, color='green', alpha=0.7)
    ax3.set_title('Histogram of residual values')
    
    return ax1, ax2, ax3
class model:
    
    def __init__(self, model):
        self.model = model
        self.x_train = None
        self.y_train = None
        self.x_test = None
        self.y_test = None
        self.y_pred_train = None
        self.y_pred_test = None
        self.train_score = None
        self.test_score = None
        self.train_score_log = None
        self.test_score_log = None
    
    def data_split(self, x, y, test_size):
        self.x_train, self.x_test, self.y_train, self.y_test = train_test_split(x, y, test_size=test_size)
    
    def score_reg(self):
        return self.train_score, self.test_score
    
    def score_log(self):
        self.train_score_log = metrics.r2_score(np.exp(self.y_train), np.exp(self.y_pred_train))
        self.test_score_log = metrics.r2_score(np.exp(self.y_test), np.exp(self.y_pred_test))
        return self.train_score_log, self.test_score_log
    
    def data_frame_convert(self):
        df_train = pd.DataFrame({'y_pred': self.y_pred_train, 'y_real': self.y_train})
        df_test = pd.DataFrame({'y_pred_test': self.y_pred_test, 'y_real_test': self.y_test})
        return self.train_score, self.test_score, df_train, df_test

    def data_frame_convert_log(self):
        df_train = pd.DataFrame({'y_pred': np.exp(self.y_pred_train), 'y_real': np.exp(self.y_train)})
        df_test = pd.DataFrame({'y_pred_test': np.exp(self.y_pred_test), 'y_real_test': np.exp(self.y_test)})
        return self.train_score_log, self.test_score_log, df_train, df_test
    
    def fit_model(self, x, y, test_size):
        self.data_split(x, y, test_size)
        self.model = self.model.fit(self.x_train, self.y_train)
        self.train_score = self.model.score(self.x_train, self.y_train)
        self.test_score = self.model.score(self.x_test, self.y_test)
        self.y_pred_train = self.model.predict(self.x_train)
        self.y_pred_test = self.model.predict(self.x_test)
    
def model_iterations(n, x, y, model_arg, log_bool=False):
    training_scores = [None]*n
    testing_scores = [None]*n

    for i in range(n):
        new_model = model(model_arg)
        new_model.fit_model(x, y, 0.3)
        training_scores[i], testing_scores[i] = new_model.score_reg() if not log_bool else new_model.score_log()

    print 'Mean Train Score:', np.mean(training_scores)
    print 'Mean Test Score:', np.mean(testing_scores)
    return new_model

In [32]:
def median_absolute_errors(x, y, log_bool=None):
    reg_params = 10.**np.linspace(-10, 5, 10)
    models = [ RidgeCV(alphas=reg_params, cv=5)]
    model_labels = np.array([ 'RidgeCV'])
    model_errors = np.array([])

    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.35, random_state=15)

    for model in models:
        model.fit(X_train, y_train)
        if not log_bool:
            model_err = metrics.median_absolute_error((y_test), model.predict(X_test))
            model_errors = np.append(model_errors, model_err)
        else:
            model_err = metrics.median_absolute_error(np.exp(y_test), np.exp(model.predict(X_test)))
            model_errors = np.append(model_errors, model_err)
    
    model_position = np.arange(model_errors.shape[0])
    models_sorted = np.argsort(model_errors)
    for i, model in enumerate(model_labels):
        print 'Model {} Results: {}'.format(model_labels[i], model_errors[i])

    plt.figure(figsize=(10,8))
    plt.bar(model_position, model_errors[models_sorted], align='center')
    plt.xticks(model_position, model_labels[models_sorted])
    plt.xlabel('Estimator')
    plt.ylabel('Median Absolute Error')
    plt.show()

In [33]:
data = pd.read_csv('../datasets/listings_clean.csv')
data.head()
# split into x and y (note that we do not include id and host_id as predictors)
x = data.iloc[:, 2:-2]
y = data.iloc[:, -2]
y_log = data.iloc[:, -1]

In [44]:
data.head(5)

,id,host_id,accommodates,bathrooms,bedrooms,beds,minimum_nights,availability_30,number_of_reviews,host_listing_count,...,50-59,60-69,70-79,80-84,85-89,90-94,95-100,No Reviews,price,price_log
0,1069266,5867023,-0.520266,-0.331542,-0.407402,-0.493039,0.173906,0.390393,2.716107,-0.355961,...,0,0,0,0,1,0,0,0,160.0,5.075174
1,2061725,4601412,-0.520266,-0.331542,-0.407402,0.381672,0.173906,-0.965897,1.295605,0.933455,...,0,0,0,0,0,0,1,0,58.0,4.060443
2,44974,198425,-0.520266,-0.331542,-0.407402,-0.493039,2.889531,-1.205242,0.822104,-0.355961,...,0,0,0,0,0,0,1,0,185.0,5.220356
3,4701675,22590025,-0.520266,-0.331542,-0.407402,0.381672,-0.601986,1.108429,-0.493176,-0.355961,...,0,0,0,0,0,0,1,0,195.0,5.273000
4,68914,343302,1.690892,-0.331542,1.266328,1.256383,-0.214040,-0.407424,0.295992,0.073844,...,0,0,0,0,0,0,1,0,165.0,5.105945


In [73]:
reg_params = 10.**np.linspace(-10, 5, 10)
RidgeCV_model = RidgeCV(alphas=reg_params, fit_intercept=True, cv=5)

In [74]:
RidgeCV_model.fit(x,y_log)

RidgeCV(alphas=array([  1.00000e-10,   4.64159e-09,   2.15443e-07,   1.00000e-05,
         4.64159e-04,   2.15443e-02,   1.00000e+00,   4.64159e+01,
         2.15443e+03,   1.00000e+05]),
    cv=5, fit_intercept=True, gcv_mode=None, normalize=False, scoring=None,
    store_cv_values=False)

In [168]:
sample = results_nona.sample(frac=0.4,axis=0)
len(sample)

1092

In [169]:
# some of the id's in the sample can't be found. So at the end we readjust the sample dataframe too so they have the same entries
sample_variables=data.loc[data['id'].isin(sample['listing_id'])]
sample_variables.head(5)
sample_variables.shape
sample = sample.loc[sample['listing_id'].isin(sample_variables['id'])]

In [170]:
sample_variables.shape

(958, 226)

In [171]:
X_sample = sample_variables.iloc[:, 2:-2]
#note y_test is the un-exponentiated price

In [172]:
new_predictions = sample.copy()
new_predictions.loc[:,0:7]=0

In [173]:
new_predictions['Mon']=np.exp(RidgeCV_model.predict(X_sample))
for i in b[1:]:
    new_predictions[i]=np.exp(RidgeCV_model.predict(X_sample))*multiplier[i]

In [174]:
new_predictions

,Mon,Tue,Wed,Thu,Fri,Sat,Sun,listing_id
875,38.752293,38.744570,38.745943,38.766520,39.939311,39.941000,38.788213,4526313.0
787,117.820157,117.796677,117.800850,117.863412,121.429096,121.434229,117.929366,2097795.0
2093,93.586445,93.567795,93.571110,93.620804,96.453083,96.457161,93.673192,3409586.0
2721,95.829400,95.810303,95.813698,95.864583,98.764742,98.768917,95.918226,3027838.0
2007,246.199927,246.150863,246.159585,246.290316,253.741256,253.751982,246.428133,2893512.0
1795,203.023346,202.982886,202.990078,203.097883,209.242137,209.250981,203.211531,4596378.0
1030,199.235418,199.195713,199.202771,199.308564,205.338181,205.346861,199.420092,5110.0
1623,134.245922,134.219168,134.223924,134.295208,138.357997,138.363845,134.370356,2563997.0
1662,163.660015,163.627400,163.633198,163.720101,168.673071,168.680201,163.811714,1814541.0
2089,151.812414,151.782160,151.787538,151.868150,156.462567,156.469181,151.953131,2500642.0


In [175]:
#sample = results_multiplier.sample(frac=0.2,axis=0)
model_err = metrics.median_absolute_error(sample.iloc[:,:-1].values.flatten(), new_predictions.iloc[:,:-1].values.flatten())

In [176]:
model_err

72.1070818480257

## Using ARIMA time series models for future forecasting